### Online machine learning
- Incremental learning experiments

In [ ]:
import os
import re
from zipfile import ZipFile
from pprint import pprint
from collections import defaultdict

import sys
sys.path.append('../')
from vibrodiagnostics import (
    mafaulda, selection, discovery, models
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import functools
from sklearn import metrics as skmetrics
import river
from river import (
    anomaly,
    cluster,
    neighbors,
    preprocessing,
    evaluate,
    stream,
    metrics,
    feature_selection,
    stats
)
import tsfel


PATH_PREFIX = '../../datasets/'
FEATURES_PATH =  os.path.join(PATH_PREFIX, 'features_data')

DATASET_PATH = os.path.join(PATH_PREFIX, 'MAFAULDA.zip')
MAFAULDA_METADATA = os.path.join(FEATURES_PATH, 'mafaulda_metadata.csv')

TD_FEATURES = os.path.join(FEATURES_PATH, selection.TIME_FEATURES_PATH)
FD_FEATURES = os.path.join(FEATURES_PATH, selection.FREQ_FEATURES_PATH)
TD_FD_FEATURES = os.path.join(FEATURES_PATH, selection.TIME_AND_FREQ_FEATURES_PATH)

TSFEL_FEATURES = os.path.join(FEATURES_PATH, selection.TSFEL_FEATURES_PATH)
TSFEL_TD_FEATURES = os.path.join(FEATURES_PATH, selection.TSFEL_TIME_FEATURES_PATH)
TSFEL_FD_FEATURES = os.path.join(FEATURES_PATH, selection.TSFEL_FREQ_FEATURES_PATH)
TSFEL_SD_FEATURES = os.path.join(FEATURES_PATH, selection.TSFEL_STAT_FEATURES_PATH)


FEATURES_FILENAME = FD_FEATURES

### Evolution of classes in dataset
- [ ] Faults over observations (number of seen until observation)
- [ ] Fault severity
- [ ] Anomaly (classes and percentages)

In [ ]:
X, y = models.features_subset(FEATURES_FILENAME, selection.FAULT_CLASSES, ['az'], 'fault')
discovery.plot_label_occurences(y)
plt.show()

In [ ]:
X, y = models.features_subset(FEATURES_FILENAME, selection.FAULT_CLASSES, ['az'], 'anomaly')
discovery.plot_label_occurences(y)
plt.show()

### Half-space trees (Anomaly)

#### HST: Parameter = Trees

In [ ]:
X, y = models.features_subset(FEATURES_FILENAME, selection.FAULT_CLASSES, ['az'], 'anomaly')

results = []
for tree in (1, 10, 30, 50, 100, 500):
    hst = (
        preprocessing.MinMaxScaler() |
        anomaly.HalfSpaceTrees(n_trees=tree, height=3, window_size=30, seed=10)
    )
    steps = evaluate.iter_progressive_val_score(
        model=hst,
        dataset=stream.iter_pandas(X, y),
        metric=metrics.ROCAUC(),
        step=100
    )
    evolution = []
    for step in steps:
        evolution.append({
            'Step': step['Step'],
            tree: step['ROCAUC'].get()
        })

    evolution = pd.DataFrame.from_records(evolution).set_index('Step')
    results.append(evolution)

results = functools.reduce(lambda a, b: pd.merge(a, b, on=['Step'], how='inner'), results)
ax = results.plot(grid=True, figsize=(15, 4), title='Half-space Tree AUC (height = 3, window = 30)', xlabel='Observation', ylabel='AUC', marker='.')
ax.legend(title='Trees')
plt.show()
results.tail(1)

#### HST: Parameter = Window size

In [ ]:
X, y = models.features_subset(FEATURES_FILENAME, selection.FAULT_CLASSES, ['az'], 'anomaly')

results = []
for window in (1, 10, 30, 50, 100, 500):
    hst = (
        preprocessing.MinMaxScaler() |
        anomaly.HalfSpaceTrees(n_trees=10, height=3, window_size=window, seed=10)
    )
    steps = evaluate.iter_progressive_val_score(
        model=hst,
        dataset=stream.iter_pandas(X, y),
        metric=metrics.ROCAUC(),
        step=100
    )
    evolution = []
    for step in steps:
        evolution.append({
            'Step': step['Step'],
            window: step['ROCAUC'].get()
        })

    evolution = pd.DataFrame.from_records(evolution).set_index('Step')
    results.append(evolution)

results = functools.reduce(lambda a, b: pd.merge(a, b, on=['Step'], how='inner'), results)
ax = results.plot(grid=True, figsize=(15, 4), title='Half-space Tree AUC (trees = 10, height = 3)', xlabel='Observation', ylabel='AUC', marker='.')
ax.legend(title='Window size')
plt.show()
results.tail(1)

#### HST: Parameter = Height of tree

In [ ]:
X, y = models.features_subset(FEATURES_FILENAME, selection.FAULT_CLASSES, ['az'], 'anomaly')

results = []
for height in (1, 2, 3, 4, 5):
    hst = (
        preprocessing.MinMaxScaler() |
        anomaly.HalfSpaceTrees(n_trees=10, height=height, window_size=30, seed=10)
    )
    steps = evaluate.iter_progressive_val_score(
        model=hst,
        dataset=stream.iter_pandas(X, y),
        metric=metrics.ROCAUC(),
        step=100
    )
    evolution = []
    for step in steps:
        evolution.append({
            'Step': step['Step'],
            height: step['ROCAUC'].get()
        })

    evolution = pd.DataFrame.from_records(evolution).set_index('Step')
    results.append(evolution)

results = functools.reduce(lambda a, b: pd.merge(a, b, on=['Step'], how='inner'), results)
ax = results.plot(grid=True, figsize=(15, 4), title='Half-space Tree AUC (trees = 10, window = 30)', xlabel='Observation', ylabel='AUC', marker='.')
ax.legend(title='Tree height')
plt.show()
results.tail(1)

#### HST: Classification clusters

In [ ]:
def hst_visualize_classes(X, y):
    hst = (
        preprocessing.MinMaxScaler() |
        anomaly.HalfSpaceTrees(n_trees=10, height=3, window_size=30, seed=10)
    )
    y_scores = []
    for xs, ys in stream.iter_pandas(X, y):
        score = hst.score_one(xs)
        hst.learn_one(xs)
        y_scores.append(score)

    y_scores = np.array(y_scores)
    models.project_anomaly_map_plot(X, y, y_scores)


X, y = models.features_subset(FEATURES_FILENAME, selection.FAULT_CLASSES, ['az'], 'anomaly')
hst_visualize_classes(X, y)